# RWKV v5 multi-size training experiment

**Note:** This project assumes you have the rwkv-infctx conda env setup

# Basic Setup

In [ ]:
# First lets setup the various directories, and init the model
!mkdir -p ../../../../model/
!mkdir -p ../../../../datapath/
!mkdir -p ../../../../checkpoint/

In [ ]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True

EMBED_SCALE=0.01
EMBED_SCALE_LABEL=str(EMBED_SCALE).replace(".", "_")

LAYER_COUNT=6
EMBED_SIZE=2048

WANDB_PREFIX=f"[Multi-size] v5-L{LAYER_COUNT}-D{EMBED_SIZE}-E{EMBED_SCALE}"
FILENAME_PREFIX=f"v5-L{LAYER_COUNT}-D{EMBED_SIZE}-E{EMBED_SCALE_LABEL}"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))
INFERENCE_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("INFERENCE_DIR:", INFERENCE_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

In [ ]:
# Init the model
!cd "{PROJECT_DIR}/model/" && wget -nc "https://huggingface.co/rwkv-x-dev/rwkv-x-playground/resolve/main/experiment/rwkv-x-exp/multi-size-train/{FILENAME_PREFIX}-enwiki-4k-p1.pth"

In [ ]:
# Lets preload the requried dataset s
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/enwiki-4k-part2a.yaml"
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/enwiki-4k-part2b.yaml"
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/enwiki-4k-part3.yaml"

## Enwiki Stage 2a : Split training

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/enwiki-4k-part2a.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Split 2A (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-split-2a/" \
        --model.load_model="../model/{FILENAME_PREFIX}-enwiki-4k-p1.pth" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python3 export_checkpoint.py "../checkpoint/{FILENAME_PREFIX}-split-2a/last.ckpt" "../model/{FILENAME_PREFIX}-split-2a.pth" "bf16"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-split-2a.pth"

In [ ]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && \
    python3 dragon_test.py "../model/{FILENAME_PREFIX}-split-2a.pth" "cuda fp32"

## Enwiki Stage 2b : Split training

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/enwiki-4k-part2b.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Split 2B (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-split-2b/" \
        --model.load_model="../model/{FILENAME_PREFIX}-enwiki-4k-p1.pth" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python3 export_checkpoint.py "../checkpoint/{FILENAME_PREFIX}-split-2b/last.ckpt" "../model/{FILENAME_PREFIX}-split-2b.pth" "bf16"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-split-2b.pth"

In [ ]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && \
    python3 dragon_test.py "../model/{FILENAME_PREFIX}-split-2b.pth" "cuda fp32"

## Enwiki Stage 2m : Merger

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python3 model_merge.py \
        --merge-mode "average" \
        "../model/{FILENAME_PREFIX}-split-2a.pth" "../model/{FILENAME_PREFIX}-split-2b.pth" \
        "../model/{FILENAME_PREFIX}-split-merge-2m.pth"

## Enwiki Stage 3 : Baseline training

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/enwiki-4k-part3.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Split Merge P3 (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.callbacks.init_args.dirpath="../checkpoint/{FILENAME_PREFIX}-split-merge-p3/" \
        --model.load_model="../model/{FILENAME_PREFIX}-split-merge-2m.pth" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python3 export_checkpoint.py "../checkpoint/{FILENAME_PREFIX}-split-merge-p3/last.ckpt" "../model/{FILENAME_PREFIX}-split-merge-p3.pth" "bf16"
!cd "{TRAINER_DIR}" && ls -alh "../model/{FILENAME_PREFIX}-split-merge-p3.pth"

In [ ]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && \
    python3 dragon_test.py "../model/{FILENAME_PREFIX}-split-merge-p3.pth" "cuda fp32"